<a href="https://colab.research.google.com/github/hayannn/AIFFEL_STUDY/blob/main/4_8_%E1%84%87%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%8C%E1%85%B5%E1%84%8B%E1%85%A1%E1%86%AB_%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A5%E1%86%A8%E1%84%92%E1%85%AA_%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB%E1%84%8B%E1%85%B4_HyperOpt%E1%84%85%E1%85%B3%E1%86%AF_%E1%84%8B%E1%85%B5%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%92%E1%85%A1%E1%86%AB_%E1%84%92%E1%85%A1%E1%84%8B%E1%85%B5%E1%84%91%E1%85%A5_%E1%84%91%E1%85%A1%E1%84%85%E1%85%A1%E1%84%86%E1%85%B5%E1%84%90%E1%85%A5_%E1%84%90%E1%85%B2%E1%84%82%E1%85%B5%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08. 베이지안 최적화 기반의 HyperOpt를 이용한 하이퍼 파라미터 튜닝
### 베이지안 최적화 개요
- 목적 함수식을 제대로 알 수 없는 블랙 박스 형태의 함수에서 최대 또는 최소 함수 반환 값을 만드는 최적 입력값을 가능한 적은 시도를 통해 빠르고 효과적으로 찾아주는 방식
- 베이지안 확률에 기반을 두고 있는 최적화 기법
- 베이지안 확률이 새로운 사건의 관측이나 새로운 샘플 데이터를 기반으로 사후 확률을 개선해나가듯, 베이지안 최적화는 새로운 데이터를 입력받았을 때 최적 함수를 예측하는 사후 모델을 개선해나가며 최적 함수 모델을 만들어냄

<br>

### 베이지안 최적화 구성의 2가지 중요 요소
- 대체 모델(Surrogate Model)
  - 획득 함수로부터 최적 함수를 예측할 수 있는 입력값을 추천받은 뒤, 이를 기반으로 최적 함수 모델을 개선
- 획득 함수(Acquisition Function)
  - 획득 함수는 개선된 대체 모델을 기반으로 최적 입력값 계산
- 베이지안 최적화를 하이퍼 파라미터 튜닝에 사용할 때에는 입력값이 하이퍼 파라미터가 됨
  - 즉, 대체 모델은 획득 함수가 계산한 하이퍼 파라미터를 입력받으며 점차 개선, 개선된 대체 모델을 기반으로 획득 함수는 더 정확한 하이퍼 파라미터를 계산

<br>

### 베이지안 최적화 단계
#### Step 1
- 최초에는 랜덤하게 하이퍼 파라미터들을 샘플링해 성능 결과 관측
- 검은색 원은 특정 하이퍼 파라미터가 입력되었을 경우 관측된 성능 지표 결과값을 뜻함
- 주황색 사선은 찾아야 할 목표 최적함수 <br>
<img width="332" alt="image" src="https://github.com/user-attachments/assets/974e5207-3028-41ba-9ba0-82ca8939768c">

<br>

#### Step 2
- 관측된 값을 기반으로 대체 모델은 최적함수를 추정
- 파란색 실선은 대체 모델이 추정한 최적 함수
- 옅은 파란색 영역은 예측된 함수의 신뢰 구간 = 추정된 함수의 결과값 오류 편차 = 추정함수의 불확실성
- 최적 관측값은 y축 value에서 가장 높은 값을 가질 때의 하이퍼 파라미터 <br>
<img width="349" alt="image" src="https://github.com/user-attachments/assets/61576ad8-6b4b-4dc6-8b0e-efbbfc328101">

<br>

#### Step 3
- 추정된 최적 함수를 기반으로 획득 함수(Acquisition Function)는 다음 관측을 할 하이퍼 파라미터 값 계산
- 획득 함수는 이전 최적 관측값보다 더 큰 최댓값을 가질 가능성이 높은 지점을 찾아 다음에 관측할 하이퍼 파라미터를 대체 모델에 전달 <br>
<img width="349" alt="image" src="https://github.com/user-attachments/assets/078be85e-1fb3-49bd-9cf7-adc93280a59a">

<br>

#### Step 4
- 획득 함수로부터 전달된 하이퍼 파라미터를 수행해 관측된 값을 기반으로 대체 모델은 갱신되어 다시 최적 함수 예측 추정 <br>
<img width="331" alt="image" src="https://github.com/user-attachments/assets/b08b59e4-c124-4a08-aaa1-3ecfd8fe7dd5">

> Step 3와 Step 4를 특정 횟수만큼 반복 -> 대체 모델의 불확실성 개선, 점차 정확한 최적 함수 추정이 가능해짐

- 대체 모델
  - 최적 함수를 추정할 때 다양한 알고리즘 사용 가능
  - 일반적으로 가우시안 프로세스(Gaussian Process) 적용
  - HyperOpt는 가우시안 프로세스가 아닌 트리 파르젠 Estimator(TPE, Tree-structure Parzen Estimator) 사용

<br>

### HyperOpt 사용하기
- 주요 로직
  - 입력 변수명과 입력값의 검색 공간(Search Space) 설정
  - 목적 함수(Objective Function) 설정
  - 목적 함수의 반환 최솟값을 가지는 최적 입력값을 유추

- 유의점
  - 다른 패키지와는 달리 목적 함수 반환값의 최댓값이 아닌 최솟값을 가지는 최적 입력값을 유추한다는 것

In [ ]:
#pip install hyperopt

In [1]:
import hyperopt

print(hyperopt.__version__)

0.2.7


In [ ]:
from hyperopt import hp

# -10 ~ 10까지 1간격을 가지는 입력 변수 x와 -15 ~ 15까지 1간격으로 입력 변수 y 설정.
search_space = {'x': hp.quniform('x', -10, 10, 1), 'y': hp.quniform('y', -15, 15, 1) }

### 입력값의 검색 공간을 제공하는 대표 함수
- `hp.quniform(label, low, high, q)` : label로 지정된 입력값 변수 검색 공간을 최솟값 low에서 최댓값 high까지 q 간격으로 설정
- `hp.quniform(label, low, high)` : 최솟값 low에서 최댓값 high까지 정규 분포 형태의 검색 공간 설정
- `hp.randint(label, upper)` : 0부터 최댓값 upper까지 random한 정숫값으로 검색 공간 설정
- `hp.loguniform(label, low, high)` : exp(uniform(low, high)) 값 반환, 반환값의 log 변환된 값은 정규 분포 형태를 가지는 검색 공간 설정
- `hp.choice(label, options)` : 검색값이 문자열 또는 문자열과 숫자가 섞여 있을 경우 설정
  - Options : 리스트나 튜플 형태로 제공
  - hp.choice('tree_criterion',['gini', 'entropy'])와 같은 설정 -> 입력 변수 tree_criterion 값을 'gini'와 'entropy'로 설정해 입력


<br>

### 목적 함수 생성
- 반드시 변수값과 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환하는 구조로 만들어져야 함

In [ ]:
from hyperopt import STATUS_OK

# 목적 함수 생성 -> 변숫값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y

    return retval

- fmin() 함수의 주요 인자
  - fn, space, algo, max_evals, trials, rstate

In [ ]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5
               , trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_01)

100%|██████████| 5/5 [00:00<00:00, 544.05trial/s, best loss: -224.0]
best: {'x': -4.0, 'y': 12.0}


- best_01 변수값 확인
  - 입력 변수 x의 공간 -10 ~ 10, y의 공간 -15 ~ 15
  - 목적 함수 반환값을 $x^2 - 20y$로 설정
  - x는 0에 가까울수록, y는 15에 가까울수록 반환값이 최소로 근사됨

<br>

In [ ]:
trial_val = Trials()

# max_evals를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|██████████| 20/20 [00:00<00:00, 221.72trial/s, best loss: -296.0]
best: {'x': 2.0, 'y': 15.0}


- 그리드 서치 : 순차적으로 x, y 변수값을 입력해 최소 함수 반환값을 찾는다면 최대 21x31 = 651회의 반복 필요
- 베이지안 최적화 : 20회의 반복만으로 일정 수준의 최적값 근사

<br>

In [ ]:
# fmin( )에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨
# 리스트 내부의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값} 와 같은 딕셔너리임.
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [ ]:
# Trials 객체의 vals 속성에 {'입력변수명':개별 수행 시마다 입력된 값 리스트} 형태로 저장됨.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


In [ ]:
import pandas as pd

# results에서 loss 키값에 해당하는 밸류들을 추출하여 list로 생성.
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성.
result_df = pd.DataFrame({'x': trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses': losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### HyperOpt를 이용한 XGBoost 하이퍼 파라미터 최적화

In [ ]:
# 아래 코드는 이전에 수록된 코드라 책에는 싣지 않았습니다.
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

- 순서
  - 적용해야 할 하이퍼 파라미터와 검색 공간 설정
  - 목적 함수에서 XGBoost를 학습 후에 예측 성능 결과를 반환값으로 설정
  - fmin() 함수에서 목적 함수를 하이퍼 파라미터 검색 공간의 입력값들을 사용해 최적 예측 성능 결과를 반환하는 최적 입력값들을 결정하는 것

- 주의
  - 특정 하이퍼 파라미터들은 정숫값만 입력은 받는데, HyperOpt는 입력값 반환값이 모두 실수형 -> 형변환 필요
  - HyperOpt의 목적 함수는 최솟값을 반환할 수 있도록 최적화해야해서 성능값이 클수록 좋은 성능 지표일 경우 -1을 곱해줘야 함

In [ ]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )

In [ ]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색.
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
                   }

- 정수형 형변환
- HyperOpt의 목적 함수는 최솟값을 반환할 수 있도록 최적화
  - 정확도와 같이 값이 클수록 좋은 성능 지표일 경우 -1을 곱한 뒤 반환 -> 더 큰 성능 지표가 더 작은 반환값이 되도록 만들어줘야 함


In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형임.
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'],
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)

    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}


In [ ]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)


100%|██████████| 50/50 [00:22<00:00,  2.22trial/s, best loss: -0.9670616939700244]
best: {'colsample_bytree': 0.5026505409854508, 'learning_rate': 0.19828428817109603, 'max_depth': 15.0, 'min_child_weight': 1.0}


- best 출력 결과
  - 정수형 하이퍼 파라미터인 max_depth, min_child_weight가 실수형 값으로 도출되었음에 유의
<br>

- fmin()으로 추출된 최적 하이퍼 파라미터를 직접 XGBClassifier에 인자로 입력하기 전
  - 정수형 하이퍼 파라미터는 정수형으로 형변환
  - 실수형 하이퍼 파라미터는 소수점 5자리까지만 변환 후 확인

In [ ]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
    int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree:0.50265, learning_rate:0.19828, max_depth:15, min_child_weight:1


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
xgb_wrapper = XGBClassifier(n_estimators=400,
                            learning_rate=round(best['learning_rate'], 5),
                            max_depth=int(best['max_depth']),
                            min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss',
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

- 도출된 최적 하이퍼 파라미터들을 이용해 XGBCalssifier를 재학습한 후, 성능 평가 결과 확인
  - XGBoost의 조기 중단을 검증 데이터 세트로 활용
  - n_estimators는 400으로 증가
  - get_clf_eval()를 이용해 성능 평가

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# XGBoost에서 사용할 데이터셋 준비
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test)

# 파라미터 설정
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'learning_rate': round(best['learning_rate'], 5),
    'max_depth': int(best['max_depth']),
    'min_child_weight': int(best['min_child_weight']),
    'colsample_bytree': round(best['colsample_bytree'], 5)
}

# 평가할 데이터 설정
evals = [(dtrain, 'train'), (dval, 'eval')]

# XGBoost 훈련
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=400,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=True
)

# 예측
preds = model.predict(dtest)

# 확률 값을 0 또는 1로 변환 (이진 분류를 위한 threshold 설정)
preds_binary = (preds > 0.5).astype(int)  # 확률이 0.5 이상이면 1, 그렇지 않으면 0

# 성능 평가
get_clf_eval(y_test, preds_binary, preds)

[0]	train-logloss:0.51230	eval-logloss:0.55948
[1]	train-logloss:0.40697	eval-logloss:0.47575
[2]	train-logloss:0.32809	eval-logloss:0.41760
[3]	train-logloss:0.27049	eval-logloss:0.37988
[4]	train-logloss:0.22671	eval-logloss:0.34438
[5]	train-logloss:0.19305	eval-logloss:0.31788
[6]	train-logloss:0.16589	eval-logloss:0.30717
[7]	train-logloss:0.14300	eval-logloss:0.30056
[8]	train-logloss:0.12355	eval-logloss:0.29707
[9]	train-logloss:0.10737	eval-logloss:0.28298
[10]	train-logloss:0.09355	eval-logloss:0.26801
[11]	train-logloss:0.08250	eval-logloss:0.26738
[12]	train-logloss:0.07357	eval-logloss:0.26220
[13]	train-logloss:0.06572	eval-logloss:0.26240
[14]	train-logloss:0.05892	eval-logloss:0.25570
[15]	train-logloss:0.05247	eval-logloss:0.25186
[16]	train-logloss:0.04804	eval-logloss:0.24924
[17]	train-logloss:0.04336	eval-logloss:0.24016
[18]	train-logloss:0.03953	eval-logloss:0.23794
[19]	train-logloss:0.03585	eval-logloss:0.23625
[20]	train-logloss:0.03349	eval-logloss:0.24089
[2

- 하이퍼 파라미터를 튜닝하지 않은 결과보다는 약간 좋은 성능 도출
  - 다만, 작은 데이터로 실행했기 때문에 fmin() 함수 인자의 rstate 값을 변경하는 경우 불안정한 성능 결과를 보일 수도 있음